In [7]:
"""
Python script to scrape Friktion User Data from Bitquery GraphQL API.

"""

import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists


class MyPortfolio:
    """
    Python ETL script for Friktion user portfolio data. Currently supported Instruction Names:
        - Deposit
        - CancelPendingDeposit
        - Withdrawal
        - CancelPendingWithdrawal
        - ClaimPendingWithdrawal
    """

    def __init__(
        self,
        date_start,
        date_end,
        ix_fname="./friktion_ix.csv",
        deposit_fname="./friktion_deposit.csv",
        deposit_cxl_fname="./friktion_deposit_cxl.csv",
        withdraw_fname="./friktion_withdraw.csv",
        withdraw_cxl_fname="./friktion_withdraw_cancel.csv",
        withdraw_claim_fname="./friktion_claim_withdrawal.csv",
        batch_size_days=2,
        batch_size_xfers=90,
        skip_ix_scrape=False
    ):
        """
        :ix_fname:              output csv for instructions
        :deposit_fname:         output csv for deposits
        :deposit_cxl_fname:     output csv for deposit cancels
        :withdraw_fname:        output csv for withdrawals
        :withdraw_cxl_fname:    output csv for withdrawal cancels
        :withdraw_claim_fname:  output csv for claiming pending withdrawal
        :batch_size_days:       batch size in days for query to keep query < 10k symbols. Use bigger steps for larger data.
        :batch_size_transfers:  batch size transactions for query to keep query < 8kb

        """
        self.volt_program = "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"
        self.date_start = date_start
        self.date_end = date_end
        self.ix_fname = ix_fname
        self.deposit_fname = deposit_fname
        self.deposit_cxl_fname = deposit_cxl_fname
        self.withdraw_fname = withdraw_fname
        self.withdraw_cxl_fname = withdraw_cxl_fname
        self.withdraw_claim_fname = withdraw_claim_fname
        self.batch_size_days = batch_size_days
        self.batch_size_xfers = batch_size_xfers
        self.skip_ix_scrape = skip_ix_scrape
        
        self.df_ix = pd.read_csv(ix_fname) if skip_ix_scrape else [] 
        self.friktion_metadata = self.get_friktion_snapshot()

    ########################################################################################################
    ####################################          Queries             ######################################
    ########################################################################################################

    @property
    def ix_query(self):
        return """
            query MyQuery {
              solana {
                instructions(
                  time: {between: ["%s", "%s"]}
                  success: {is: true}
                  programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
                  options: {limit: 8700}
                ) {
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  transaction {
                    signature
                    feePayer
                  }
                  data {
                    base58
                  }
                }
              }
            }
        """

    @property
    def xfer_query(self):
        return """
            query MyQuery {
              solana(network: solana) {
                transfers(
                  signature: {in: [%s]}
                  options: {limit: 2000}
                ) {
                  instruction {
                    action {
                      name
                    }
                    callPath
                  }
                  amount(success: {is: true})
                  transaction {
                    signer
                    signature
                  }
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  currency {
                    name
                    address
                  }
                  sender {
                    address
                    mintAccount
                  }
                  receiver {
                    address
                  }
                }
              }
            }
        """

    ########################################################################################################
    ################################          Helper Functions             #################################
    ########################################################################################################

    # TODO: Add retry logic to this in case of hangups.
    @staticmethod
    def run_query(query, retries=10):
        """
        Query graphQL API.

        If timeerror
        """
        headers = {"X-API-KEY": "BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4"}

        retries_counter = 0
        try:
            request = requests.post(
                "https://graphql.bitquery.io/", json={"query": query}, headers=headers
            )
            result = request.json()
            # print(dir(request.content))
            # Make sure that there is no error message
            # assert not request.content.errors
            assert "errors" not in result
        except:
            while (
                (request.status_code != 200
                or "errors" in result)
                and retries_counter < 10
            ):
                print(datetime.now(), f"Retry number {retries_counter}")
                if "errors" in result:
                    print(result["errors"])
                print(datetime.now(), f"Query failed for reason: {request.reason}. sleeping for {150*retries_counter} seconds and retrying...")
                time.sleep(150*retries_counter)
                request = requests.post(
                    "https://graphql.bitquery.io/",
                    json={"query": query},
                    headers=headers,
                )
                retries_counter += 1
            if retries_counter >= retries:
                raise Exception(
                    "Query failed after {} retries and return code is {}.{}".format(
                        retries_counter, request.status_code, query
                    )
                )
        return request.json()


    @staticmethod
    def batch_iterable(iterable, n=1):
        """
        Takes in an iterable and returns an iterable of iterables with len==x
        """
        idxs = []
        l = len(iterable)
        for idx in range(0, l, n):
            idxs.append(iterable[idx : min(idx + n, l)])
        return idxs

    def format_txs_for_query(self, tx_signatures):
        """
        Batches a list of transactions into a list of string formatted transactions for querying.
        Each of these strings contain (n=self.batch_size_xfers) unique transaction IDs.
        """
        batched_signatures = self.batch_iterable(tx_signatures, self.batch_size_xfers)

        def format_txs(x):
            return str(x)[1:-1].replace("'", '"').replace("\n", "")

        tx_strs = list(map(format_txs, batched_signatures))

        return tx_strs

    def get_existing_df(self, fname):
        """
        Create output file if doesn't exist and returns a DataFrame
        """
        if fname and exists(fname):
            return pd.read_csv(fname)
        else:
            return pd.DataFrame()

    @staticmethod
    def instruction_match(instructionData):
        """
        Match 8-bit instruction identifier with the corresonding instructionType
        """
        if not instructionData or len(instructionData) < 8:
            return False

        instructionDescriptor = instructionData[:8]

        if instructionDescriptor == "PcB3tF1K":
            return "Withdraw"
        elif instructionDescriptor == "WuE7Hjns":
            return "Deposit"
        elif instructionDescriptor == "V8cW2nMq":
            return "CancelPendingDeposit"
        elif instructionDescriptor == "dxUbSCWk":
            return "CancelPendingWithdrawal"
        elif instructionDescriptor == "WcTWQsnk":
            return "ClaimPendingWithdrawal"
        else:
            return "Unclassified"

    def get_friktion_snapshot(self):
        """
        Load Friktion Metadata for Volt/Symbol Mapping to join to normal data

        """
        try:
            return pd.DataFrame(
                dict(
                    json.loads(
                        requests.get(
                            "https://friktion-labs.github.io/mainnet-tvl-snapshots/friktionSnapshot.json"
                        ).content
                    )
                )["allMainnetVolts"]
            )[
                [
                    "globalId",
                    "vaultAuthority",
                    "shareTokenMint",
                    "depositTokenSymbol",
                    "depositTokenCoingeckoId",
                ]
            ]
        except Exception as e:
            print(datetime.now(), "Snapshot Data Invalid")
            traceback.print_exc()

    ########################################################################################################
    ################################          Data Retrieval             ###################################
    ########################################################################################################

    def get_ix(self, date_start, date_end):
        """
        Runs graphql instruction query for one date range.
        """
        print(date_start, date_end)
        query = self.ix_query % (date_start, date_end)
        print(
            datetime.now(),
            "retrieving instructions for {} to {}".format(date_start, date_end),
        )
        result = self.run_query(query)

        # convert GraphQL json to pandas dataframe
        try:
            df = pd.json_normalize(result["data"]["solana"]["instructions"])
        except:
            print(result)
            traceback.print_exc()
            raise Exception(datetime.now(), "Empty Results... Try Again")

        print(datetime.now(), df.shape[0], "instructions retrieved")

        df = df.rename(
            columns={
                "block.timestamp.iso8601": "timestamp",
                "log.consumed": "computeUnits",
                "transaction.signature": "txSignature",
                "transaction.feePayer": "txSigner",
                "data.base58": "instructionData",
            }
        )
        return df

    def get_ix_batch(self):
        """
        Batch the instruction retrieval. Save the shit Drop duplicates.

        """
        # Batch the days up nice and good so the graphql API calls don't bitch
        dates_batched = pd.date_range(self.date_start, self.date_end, freq=f"{self.batch_size_days}D")
        dates_batched = [
            str(x.isoformat())
            for x in dates_batched.append(pd.DatetimeIndex([self.date_end]))
        ]
        date_ranges = list(zip(dates_batched, dates_batched[1:]))
        ixs = []
        for date_range in date_ranges:
            assert len(date_range) == 2
            data = self.get_ix(date_range[0], date_range[1])
            ixs.append(data)

        df_ix = pd.concat(ixs, ignore_index=False)
        df_ix["instructionType"] = df_ix.instructionData.apply(
            lambda x: self.instruction_match(x)
        )

        # Store df_ix before we write it to the DataFrame so we avoid getting xfers for every single ix 
        self.df_ix = df_ix.drop_duplicates(["txSignature", "instructionType"]).reset_index(drop=True)
        print(datetime.now(), "final instruction data size: ", df_ix.shape[0])

        # Save new data to file
        df_old = self.get_existing_df(self.ix_fname)
        df = df_ix.append(df_old).reset_index(drop=True)
        df = df.drop_duplicates(["txSignature", "instructionType"])
        df.to_csv(self.ix_fname, index=False)
        print(datetime.now(), "wrote instruction data to csv...")

    def get_batched_xfers(self, instructionType, fname):
        """
        Get all transfers corresponding to a specific instructionType from Graphql query.
        Batch these queries up b/c the string size is
        too large (curse GraphQL for not supporting joins)

        :instructionType: String corresponding to the instruction type of each query.
        :fname: Name of where the old df is stored

        """

        temp = self.df_ix.query(f"instructionType == '{instructionType}'")

        if temp.empty:
            print(
                datetime.now(), "instructionType was not found in the data... breaking"
            )
            return

        tx_signatures = list(temp["txSignature"].unique())
        tx_strs = self.format_txs_for_query(tx_signatures)
        print(datetime.now(), len(tx_strs), "signature batches required...")
        xfers = []

        for i, tx_str in enumerate(tx_strs):
            query = self.xfer_query % (tx_str)
            result = self.run_query(query)
            # TODO: Clean up this duplicated logic
            try:
                df = pd.json_normalize(result["data"]["solana"]["transfers"])
            except:
                print(result)
                traceback.print_exc()
                raise Exception("Empty Results... Try Again")
            xfers.append(df)
            print(datetime.now(), df.shape[0], "transfers scraped in batch %d" % i)

        df_xfer = pd.concat(xfers, ignore_index=False)
        df_xfer = df_xfer.rename(
            columns={
                "block.timestamp.iso8601": "timestamp",
                "instruction.action.name": "instructionAction",
                "instruction.callPath": "instructionOrder",
                "transaction.signer": "userAddress",
                "transaction.signature": "txSignature",
                "currency.name": "currencyName",
                "receiver.address": "receiverAddress",
                "sender.address": "senderAddress",
                "currency.decimals": "currencyDecimals",
                "currency.address": "currencyAddress",
                "sender.mintAccount": "senderTokenMint",
            }
        )

        print(datetime.now(), df_xfer.shape[0], "transfers retrieved")

        df_old = self.get_existing_df(fname)
        df_final = df_old.append(df_xfer, ignore_index=True).sort_values(
            "instructionOrder"
        )

        return df_final

    def parse_deposits(self):
        instructionType = "Deposit"
        instructionAction = "transfer"
        tx_merge_key = "receiverAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.deposit_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_withdrawal(self):
        instructionType = "Withdraw"
        instructionAction = "burn"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_deposit_cancel(self):
        instructionType = "CancelPendingDeposit"
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.deposit_cxl_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_withdrawal_cancel(self):
        instructionType = "CancelPendingWithdrawal"
        instructionAction = "mintTo"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_cxl_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_claim_withdrawal(self):
        instructionType = "ClaimPendingWithdrawal"
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.withdraw_claim_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_base(
        self,
        instructionType,
        instructionAction,
        tx_merge_key,
        meta_merge_key,
        output_file,
    ):
        """
        generalized method for parsing transfer data.

        1. Call get_batched_xfers()
        2. for each unique txSignature, find the xfer matching to the last instance of the instructionAction
        3. For SOL CC vaults, need an extra step and query for instructionAccount to find vaultAuthority.
        3. Join it to the friktion metadata based using tx_merge_key and meta_merge_key
        4. Drop extraneous rows
        5. Save the file to the output_file

        :instructionType: Type of instruction listed out in the instruction_match() method
        :instructionAction: type of transfer we are matching towards
        :tx_merge_key: what key in the xfer dataFrame do we want to merge on
        :meta_merge_key: what key in the metadata dataFrame we want to merge on.
        :output_file: as name suggests
        """
        print(
            datetime.now(),
            "Parsing transfers for instructionType: %s" % instructionType,
        )
        df = self.get_batched_xfers(instructionType, output_file)

        # Target only wrapped SOL entries for SOL vaults
        df = df.query('currencyName != "Solana"')

        df = (
            df.query('instructionAction=="{}"'.format(instructionAction))
            .groupby("txSignature")
            .last()
            .reset_index()
        )

        # Join tables and get rid of extraneous columns from metadata.
        df = pd.merge(
            df,
            self.friktion_metadata,
            how="left",
            left_on=tx_merge_key,
            right_on=meta_merge_key,
            suffixes=("", "_drop"),
        )
        df.drop([col for col in df.columns if "drop" in col], axis=1, inplace=True)

        # Tag the row with the instructionType
        df["userAction"] = instructionType

        df.drop_duplicates().to_csv(output_file, index=False)
        print(datetime.now(), "{} data size: {}".format(instructionType, df.shape[0]))

    def parse_all(self):
        if not self.skip_ix_scrape:
            self.get_ix_batch()
            self.parse_claim_withdrawal()
            self.parse_deposit_cancel()
            self.parse_withdrawal_cancel()
            self.parse_withdrawal()
            self.parse_deposits()

    ########################################################################################################
    #####################################          Tests             #######################################
    ########################################################################################################

    def check_fidelity(self):
        ix = pd.read_csv(self.ix_fname)
        deposits = pd.read_csv(self.deposit_fname)
        withdrawals = pd.read_csv(self.withdraw_fname)
        claim = pd.read_csv(self.withdraw_claim_fname)
        withdrawals_cxl = pd.read_csv(self.withdraw_cxl_fname)
        deposits_cxl = pd.read_csv(self.deposit_cxl_fname)

        assert ix.loc[ix.instructionType == "Deposit"].shape[0] == deposits.shape[0]
        assert ix.loc[ix.instructionType == "Withdraw"].shape[0] == withdrawals.shape[0]
        assert (
            ix.loc[ix.instructionType == "ClaimPendingWithdrawal"].shape[0]
            == claim.shape[0]
        )
        assert (
            ix.loc[ix.instructionType == "CancelPendingDeposit"].shape[0]
            == deposits_cxl.shape[0]
        )
        assert (
            ix.loc[ix.instructionType == "CancelPendingWithdrawal"].shape[0]
            == withdrawals_cxl.shape[0]
        )


# if __name__ == "__main__":
#     date_start = "2021-12-16T16:00:00Z"
#     date_end = "2022-04-01T00:00:00Z"

#     x = MyPortfolio(date_start, date_end)
#     x.parse_all()
#     x.check_fidelity()


In [8]:
date_start = "2022-03-31T00:00:00Z"
date_end = "2022-04-04T00:00:00Z"

x = MyPortfolio(date_start, date_end, skip_ix_scrape=False)

x.parse_all()

x.check_fidelity()

2022-03-31T00:00:00+00:00 2022-04-02T00:00:00+00:00
2022-04-03 22:42:55.321764 retrieving instructions for 2022-03-31T00:00:00+00:00 to 2022-04-02T00:00:00+00:00
2022-04-03 22:43:01.452340 2759 instructions retrieved
2022-04-02T00:00:00+00:00 2022-04-04T00:00:00+00:00
2022-04-03 22:43:01.453341 retrieving instructions for 2022-04-02T00:00:00+00:00 to 2022-04-04T00:00:00+00:00
2022-04-03 22:43:05.083959 805 instructions retrieved
2022-04-04T00:00:00+00:00 2022-04-04T00:00:00+00:00
2022-04-03 22:43:05.084658 retrieving instructions for 2022-04-04T00:00:00+00:00 to 2022-04-04T00:00:00+00:00
2022-04-03 22:43:09.477322 0 instructions retrieved
2022-04-03 22:43:09.499734 final instruction data size:  3564
2022-04-03 22:43:09.871592 wrote instruction data to csv...
2022-04-03 22:43:09.874655 Parsing transfers for instructionType: ClaimPendingWithdrawal
2022-04-03 22:43:09.877448 7 signature batches required...
2022-04-03 22:43:17.492074 352 transfers scraped in batch 0
2022-04-03 22:43:21.178

AssertionError: 

In [3]:
        ix = pd.read_csv("friktion_ix.csv")
#         deposits = pd.read_csv("friktion_deposit.csv")
        withdrawals = pd.read_csv("friktion_withdraw.csv")
#         claim = pd.read_csv(self.withdraw_claim_fname)
#         withdrawals_cxl = pd.read_csv(self.withdraw_cxl_fname)
#         deposits_cxl = pd.read_csv(self.deposit_cxl_fname)
        
#         print(ix)
#         print(deposits)

#         assert ix.loc[ix.instructionType=="Deposit"].shape[0] == deposits.shape[0]
#         assert ix.loc[ix.instructionType=="Withdraw"].shape[0] == withdrawals.shape[0]
#         assert ix.loc[ix.instructionType=="ClaimPendingWithdrawal"].shape[0] == claim.shape[0]
#         assert ix.loc[ix.instructionType=="CancelPendingDeposit"].shape[0] == deposits_cxl.shape[0]
#         assert ix.loc[ix.instructionType=="CancelPendingWithdrawal"].shape[0] == withdrawals_cxl.shape[0]

In [4]:
ix.timestamp.max()

'2022-04-03T23:32:20Z'

In [223]:
withdrawals

,txSignature,amount,instructionAction,instructionOrder,userAddress,timestamp,currencyName,currencyAddress,senderAddress,senderTokenMint,receiverAddress,globalId,vaultAuthority,shareTokenMint,depositTokenSymbol,depositTokenCoingeckoId,userAction
0,124pSR5mQXtZAQ3vES5QTqkZzyBiyYphVhc3CHkswcibTm...,3.803200e+02,burn,0-1,75JCoqqnK9aHwhGSTrYacwDDPxaySZBHVwxQ9WAdmRic,2022-01-01T07:47:41Z,NaN,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,75JCoqqnK9aHwhGSTrYacwDDPxaySZBHVwxQ9WAdmRic,DMxk7cTKY94j9ks84nBFb1d5H5RisWqcKheuVPzY1mHX,NaN,mainnet_income_call_sbr,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,SBR,saber,Withdraw
1,2e2oDbhVmghFLJxt7PAByVnBJSSqHf8TXFkZyPryd51YPd...,6.783181e-01,burn,2-1,CsSL2fTbEUvKTFr5Bp5u8mkPmxNSWA7NeLoJHMYku5i9,2022-01-01T07:59:40Z,NaN,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,CsSL2fTbEUvKTFr5Bp5u8mkPmxNSWA7NeLoJHMYku5i9,5ymHA7Xo45RRc82RYRE9iTuUpNxyvkuv19ktarbrp3eQ,NaN,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana,Withdraw
2,2hUfE3urXF4gXBioRKhDzPbxebVximKvPvykHMwLivq7jp...,3.000000e+03,burn,0-0,7C8f8dxxqQEZyKpHMWaQtfGckrkLgUYmfEH9tjcwB8HT,2022-01-01T06:25:14Z,NaN,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,7C8f8dxxqQEZyKpHMWaQtfGckrkLgUYmfEH9tjcwB8HT,4FMsmFpS2wN6R3BXzoGgZbEg3gSrqe7jH5ocVJyJH17N,NaN,mainnet_income_put_sol,6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,USDC,usd-coin,Withdraw
3,2jUEu8tq5xBSjfUtsAaUDyJuT7ba5wKeZuuNBdD8sLS9eq...,1.957084e-01,burn,2-1,8763yZEdiDxfigtDrNoncoRWm95ZxmUfjH3FRhx2pNaZ,2022-01-01T14:16:02Z,NaN,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,8763yZEdiDxfigtDrNoncoRWm95ZxmUfjH3FRhx2pNaZ,44dDvkDNaptGNQvNUn76xahRaMf3knKqHYsCv5mMzXj9,NaN,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana,Withdraw
4,2pZX2N78R4ZzwzG7XKLXS65CDKFu5gRBRFE27i9dgtPfkw...,1.957084e-01,burn,2-1,AzdraX1CAboGXtQuLdbuMCCUe6YexSzhCGxQfZ47mP5N,2022-01-01T14:12:43Z,NaN,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,AzdraX1CAboGXtQuLdbuMCCUe6YexSzhCGxQfZ47mP5N,6RjzqzJEBafuNrTvSwexm9KiLaJu1Zd8w5RiH3oE5xRi,NaN,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana,Withdraw
5,2pcVMhvKVWXH6FKJ8Xg1YyAA2khj6VzvxWZYgL8wWrNnWY...,1.957084e-01,burn,2-1,F9FCtdcvLEL8Dn7wL9ThEXz3EvcscX9bQzbUXQiszXzC,2022-01-01T14:14:29Z,NaN,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,F9FCtdcvLEL8Dn7wL9ThEXz3EvcscX9bQzbUXQiszXzC,64xTXckCmiKHnkD9TkuY8Znbqm8oCZzK8gGLtYxiH1Q8,NaN,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana,Withdraw
6,37nVMJcYiXzf78wFz5SxErYSFcR1JdrxEoyXZC4gEZdhCA...,1.259734e+01,burn,3-1,13kgmSo7S4wcjfsTEgsmQDunGq138kTgKhaK1jSVkE5H,2022-01-01T12:14:55Z,NaN,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,13kgmSo7S4wcjfsTEgsmQDunGq138kTgKhaK1jSVkE5H,4Y7xnCiT5UEiawK1eFKzcTmioMDkP9K39Uy7a2dL2z8E,NaN,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana,Withdraw
7,3AvMEehi3zeNz9QZJDEwq5L5fnQQkZz3NG9VSUDQNEnjFF...,2.390000e-03,burn,0-1,GFPXvPpn7oTX56Xf4bNhz52EbfLB9inhGiS141rWdGZ8,2022-01-01T08:11:42Z,NaN,GjnoPUjQiEUYWuKAbMax2cM1Eony8Yutc133wuSun9hS,GFPXvPpn7oTX56Xf4bNhz52EbfLB9inhGiS141rWdGZ8,ESshaD7sX3EWxWpavToRaYZdU6KfzPpTXVc8JU2YunYG,NaN,mainnet_income_call_eth,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,GjnoPUjQiEUYWuKAbMax2cM1Eony8Yutc133wuSun9hS,ETH,ethereum,Withdraw
8,3CW2zoTALssgttfNbcbnJpPYqStT8B8eVPGKFdCS6veAXA...,1.541407e+04,burn,1-1,AzdraX1CAboGXtQuLdbuMCCUe6YexSzhCGxQfZ47mP5N,2022-01-01T14:07:36Z,NaN,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,AzdraX1CAboGXtQuLdbuMCCUe6YexSzhCGxQfZ47mP5N,4XZm7N2gJFJTGCrNmZqH5Puytj5c9sBYta8RNGSdTpeN,NaN,mainnet_income_call_sbr,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,SBR,saber,Withdraw


In [ ]:
deposits.shape

In [196]:
ix.loc[ix.instructionType=="Deposit"].shape

(29408, 5)

In [197]:
deposits.sort_values("amount")[::-1]

,txSignature,amount,instructionAction,instructionOrder,userAddress,timestamp,currencyName,currencyAddress,senderAddress,senderTokenMint,receiverAddress,globalId,vaultAuthority,shareTokenMint,depositTokenSymbol,depositTokenCoingeckoId,userAction
7486,3FvYTiomvJTpAjs5XxGc81egpWzebGXA4sSs7VMvWB6c9G...,4.879066e+06,transfer,1-1,7xTjub4EpTLx7QLHdeJKNQsuVYkqaiGbBd4DJRYjKAYZ,2022-01-13T17:40:28Z,Saber Protocol Token,Saber2gLauYim4Mvftnrasomsv6NvAuncvMEZwcLpD1,7xTjub4EpTLx7QLHdeJKNQsuVYkqaiGbBd4DJRYjKAYZ,3JEfWY4V6sQBQXsbXeWkDVA5pqPisudwhxRbWWd76rt9,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,NaN,NaN,NaN,NaN,NaN,Deposit
11664,3zZ8TKn9gYpFhrqcx8W2HTJikwHSTZLG3GyiqbM12BWPQY...,2.998353e+06,transfer,1-1,CHbwVQQvMvhGxLPBkaL6SZuJ8f895bGHZ5BsLXbaJL3N,2022-02-05T01:47:19Z,NaN,Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip,CHbwVQQvMvhGxLPBkaL6SZuJ8f895bGHZ5BsLXbaJL3N,424dm7Lauis6QRBreU9nt8yvwe1FZqUtp6oiEAzMcttQ,AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8,NaN,NaN,NaN,NaN,NaN,Deposit
19671,5M3H96K3RJghDp5R3SdJsCyeCnRYypTXtL5PvzE6W8wS82...,2.943753e+06,transfer,1-1,GmRC6EhKtBEcKM1bjCBzamWDy3qmP5z2Kc9tYjQuc2Pn,2021-12-30T10:31:36Z,Saber Protocol Token,Saber2gLauYim4Mvftnrasomsv6NvAuncvMEZwcLpD1,GmRC6EhKtBEcKM1bjCBzamWDy3qmP5z2Kc9tYjQuc2Pn,FLEiW8R1qj2XnZ5vuJskFZ1X7umntUJV2DAgFRckcfv5,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,NaN,NaN,NaN,NaN,NaN,Deposit
1691,2GR1JARZMSRP9wEtvVg5D1tbMptBayGuhCt6BuHjevvztx...,2.904146e+06,transfer,0-0,3KNZ9i1dLNNqpBTKEkTgUQs6TNCd3bzuy6HwfoXACaRs,2022-03-07T10:10:46Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3KNZ9i1dLNNqpBTKEkTgUQs6TNCd3bzuy6HwfoXACaRs,8LrqyZvCsyBVFXDU9YbVAj6ukJ9tSKeXTmXNHWfZkGRF,BVrYZ1XpjK85kYKSd2bsQweidBJxHnf8exbpZCNcMdTQ,NaN,NaN,NaN,NaN,NaN,Deposit
25363,JBK4C7XkJKknRMD9NTmiengbiBn9rwNTdFd8MUDvbvsSHa...,2.630611e+06,transfer,1-1,3KNZ9i1dLNNqpBTKEkTgUQs6TNCd3bzuy6HwfoXACaRs,2022-03-28T21:44:12Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3KNZ9i1dLNNqpBTKEkTgUQs6TNCd3bzuy6HwfoXACaRs,8LrqyZvCsyBVFXDU9YbVAj6ukJ9tSKeXTmXNHWfZkGRF,8ENPcthqhuWVaTG3eE28zJHxm4BWzyiyCFgqZL44szTY,NaN,NaN,NaN,NaN,NaN,Deposit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15554,4eVtsrbxsB5yKwANXpWQU5kH5Qc3HAD37ak8L9uzk5GEeU...,1.000000e-09,transfer,0-0-0,GgF8AhcdiJ4hhyFaMoJ3XpQo9q7NgFnNaMQ6x6Xj3Lvt,2021-12-27T02:05:05Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,6qfyGvoUqGB6AQ7xLc4pVwFNdgJSbAMkTtKkBXhLRiV1,5Vfn9EmBYkxDFASQKLEZqDuDCT2G2tbmNrizisa4m4uL,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,NaN,NaN,NaN,NaN,NaN,Deposit
12164,45X8ZKAUZUPZBVorVhavjAgncR2KXynevXNB9sRkAwbSQH...,1.000000e-09,transfer,1-1,J1sFjvcmhAxiYS5oerp167VDCu1u76csG5bXkkgijVJn,2022-01-24T23:48:03Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,J1sFjvcmhAxiYS5oerp167VDCu1u76csG5bXkkgijVJn,EqXN26d6mKUKPXrziWxaHxF7wsDzt95vWPnFkFMe7Nok,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,NaN,NaN,NaN,NaN,NaN,Deposit
16534,4pSgwJkyp6PuitQMC5UiKoJiV1bmUUk9E7VzE73Ds9XMqb...,1.000000e-09,transfer,1-1,GFZpgo47bsmPzLentNshwHnapbryMiu1hfisng7DBbDm,2022-03-30T09:39:59Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,GFZpgo47bsmPzLentNshwHnapbryMiu1hfisng7DBbDm,5BMHymXSNFgyoGYzxx6b9yGTTq4zRwXzCkrwu2SUFs2n,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,NaN,NaN,NaN,NaN,NaN,Deposit
2386,2PXZks2t4YUZWQuDc6MNo1nLre5SgMWhumRvGx4jqemQSS...,1.000000e-09,transfer,0-0,5cvX5RGXCbX12Rso1UCTFFDdzgN9W7jMTHCDzKAthmuo,2022-01-26T21:29:48Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,5cvX5RGXCbX12Rso1UCTFFDdzgN9W7jMTHCDzKAthmuo,EFHB1NyzjTQW7hmAorGzHBoUqdVZDsGQ4jDVyPMXTvjn,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,NaN,NaN,NaN,NaN,NaN,Deposit


In [163]:
pd.options.display.width = 0

In [177]:
deposits.loc[
    deposits.vaultAuthority=="Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G"].sort_values("amount")[["timestamp", "amount", "txSignature"]]

,timestamp,amount,txSignature
7978,2022-02-25T19:56:08Z,1.000000e-09,mQPf6mdLdSpCtnmvPkKarwhMykNNp5FEzKg6yzvJbo12yG...
5862,2022-02-12T02:52:57Z,1.000000e-09,5VtrMPM6nGsWSzeWrFQ4dQFnHyixUMmK4GXF5wzyRsYp2L...
479,2022-02-03T21:05:08Z,1.000000e-09,2H4hQFTSkvWJUTr3t5f9i8hFDG1XW7Rak31WoiDLqf733t...
4532,2022-03-18T14:21:43Z,1.000000e-09,4h55mB7Hwosedtx3kttVfbLmAGdKoHkKRoSY7fhBoufYVU...
5087,2022-01-07T15:34:27Z,1.000000e-09,52jsKGk4ryoqbQCiNnZg1TcDJ81RHxq5PrptiunnjW3TJb...
...,...,...,...
2415,2022-01-12T08:06:22Z,3.638800e+03,3Rkt6Z9QHzxxrP6FhVoLA4jkcKCXJPjASDsjeUHmbbBsUd...
4309,2022-03-30T16:18:44Z,8.230664e+03,4a2KegHZpt2Mfp45FjcbFUVV1HsMHSHzyN6kfML9mJ8qrN...
4799,2022-02-01T16:45:17Z,8.586272e+03,4rbL6s8PcsHaPBBM5Sf1mgGBkQHUvVo8zxa5VhP92oU2oN...
2362,2022-02-17T09:13:40Z,1.168541e+04,3PezBkLDECULRfmgLNz92qZewjzC6oAbr7xAbstGTuV1CN...


In [176]:
deposits.loc[
    deposits.vaultAuthority=="Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G"].sort_values("amount")[["timestamp", "amount", "txSignature"]][::-1].iloc[3]["txSignature"]


'4a2KegHZpt2Mfp45FjcbFUVV1HsMHSHzyN6kfML9mJ8qrN4iQKGUWPXBetXMrGKDRtu8JuhZPC4ttBZqaiAtL3dD'

In [ ]:
depsots

In [111]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId', 'userAction'],
      dtype='object')

In [ ]:
deposits

In [28]:
# Have Price
# Have VoltTokenShare

In [27]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId'],
      dtype='object')

In [16]:
deposits.shape

(115, 16)

In [26]:
deposits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   txSignature              115 non-null    object 
 1   amount                   115 non-null    float64
 2   instructionAction        115 non-null    object 
 3   instructionOrder         115 non-null    object 
 4   userAddress              115 non-null    object 
 5   timestamp                115 non-null    object 
 6   currencyName             115 non-null    object 
 7   currencyAddress          115 non-null    object 
 8   senderAddress            115 non-null    object 
 9   senderTokenMint          115 non-null    object 
 10  receiverAddress          115 non-null    object 
 11  globalId                 115 non-null    object 
 12  vaultAuthority           115 non-null    object 
 13  shareTokenMint           115 non-null    object 
 14  depositTokenSymbol       1

In [17]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId'],
      dtype='object')

In [147]:
deposits.iloc[0]

txSignature                122tbxak1CeaRQSMZD2izGb9tXa7rSaRpyD2s3iSv4bKp3...
amount                                                              1.062448
instructionAction                                                       burn
instructionOrder                                                         1-1
userAddress                     GbYTQgbXndUJoczx23Hy3hobAReguEuddijnsRTkn8pJ
timestamp                                               2022-01-09T10:12:13Z
currencyName                                                             NaN
currencyAddress                 6UA3yn28XecAHLTwoCtjfzy3WcyQj1x13bxnH8urUiKt
senderAddress                   GbYTQgbXndUJoczx23Hy3hobAReguEuddijnsRTkn8pJ
senderTokenMint                 2ALneL69HRuthso7f4inq75g8AYBTbKHKgFB2oBgE6mh
receiverAddress                                                          NaN
globalId                                                                 NaN
vaultAuthority                                                           NaN

In [19]:
deposits.loc[deposits.depositTokenCoingeckoId.isna()].currencyAddress

Series([], Name: currencyAddress, dtype: object)

In [20]:
deposits.shape

(115, 16)

In [21]:
deposits.loc[deposits.txSignature=="24y4KRma4BUarjw4y8Rv9Dt1kZdVVSP5kVnkbu4Ra5KMs7G9Df2Vb9C1b3M8f6yCuNuhsEW9s1WR8WrSKuN5bVBX"].iloc[0]

txSignature                24y4KRma4BUarjw4y8Rv9Dt1kZdVVSP5kVnkbu4Ra5KMs7...
amount                                                                 551.0
instructionAction                                                   transfer
instructionOrder                                                         3-1
userAddress                     9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw
timestamp                                               2022-03-17T18:39:38Z
currencyName                                                     Wrapped SOL
currencyAddress                  So11111111111111111111111111111111111111112
senderAddress                   9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw
senderTokenMint                 D9QnLjbjTEBCqFZUrovoFkNW3B8gLmsJkT6PDt4HUv1H
receiverAddress                  wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf
globalId                                        mainnet_income_call_sol_high
vaultAuthority                   wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf

In [22]:
deposits.loc[deposits.txSignature=="1kx6PYTANDwsAZGZ5xoRNe51uRK68k6roF6KYDauHQMmemH4JunDBCA47oHT2J1QUbh5YJHFdoZtQECqUVLVbZp"].iloc[0]

txSignature                1kx6PYTANDwsAZGZ5xoRNe51uRK68k6roF6KYDauHQMmem...
amount                                                               0.99859
instructionAction                                                   transfer
instructionOrder                                                         1-1
userAddress                     79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT
timestamp                                               2022-03-17T19:33:26Z
currencyName                                       Wrapped Ethereum (Sollet)
currencyAddress                 2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk
senderAddress                   79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT
senderTokenMint                 3e81Tr2AGVLwPLLsUwiBSaYdGDwSxCMXmhF5ZWUuQAsn
receiverAddress                 FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp
globalId                                             mainnet_income_call_eth
vaultAuthority                  FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp

In [149]:
zz = pd.DataFrame(
                dict(
                    json.loads(
                        requests.get(
                            "https://friktion-labs.github.io/mainnet-tvl-snapshots/friktionSnapshot.json"
                        ).content
                    )
                )["allMainnetVolts"]
            )

In [150]:
zz

,globalId,vaultAuthority,shareTokenMint,depositTokenSymbol,depositTokenCoingeckoId
0,mainnet_income_call_btc,DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du,3BjcHXvyzMsjmeqE2qFLx45K4XFx3JPiyRnjJiF5MAHt,BTC,bitcoin
1,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,solana
2,mainnet_income_call_sol_high,wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,SOL,solana
3,mainnet_income_call_marinade,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,6UA3yn28XecAHLTwoCtjfzy3WcyQj1x13bxnH8urUiKt,mSOL,msol
4,mainnet_income_call_eth,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,GjnoPUjQiEUYWuKAbMax2cM1Eony8Yutc133wuSun9hS,ETH,ethereum
5,mainnet_income_call_ftt,7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K,7wDh4VCTPwx41kvbLE6fkFgMEjnqw7NpGJvQtNabCm2B,FTT,ftx-token
6,mainnet_income_call_srm,2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,SRM,serum
7,mainnet_income_call_mngo,B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ,4sTuzTYfcE2NF7zy6Sy8XhVcNLa6JQSLrx3roy97n4sD,MNGO,mango-markets
8,mainnet_income_call_socean,A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A,5VmdHqvRMbXivuC34w4Hux9zb1y9moiBEQmXDrTR1kV,scnSOL,socean-staked-sol
9,mainnet_income_call_sbr,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,SBR,saber
